# Multilayer Perceptron Regression Model (Version 1)

Regression with MLP using ScikitLearn's MLP Regressor. Feel free to adapt to using PyTorch/TensorFlow multilayer perceptron if deemed suitable.

Disclaimer: As a _suay_ kid, I can't do hyperparameter tuning, so this is just a baseline implementation with Optuna for automated hyperparameter search.

### Summary

| Techniques                     | Used / Description           |
| ------------------------------ | ---------------------------- |
| Handling Unknown Variables     | Drop Rows                    |
| Handling Categorical Variables | Drop Columns (Drop Features) |
| Handling Class Imbalance       | Not Applied                  |
| Handling Outliers              | Not Applied                  |

### Results

| Metric                 | Value  |
| ---------------------- | ------ |
| RMSE (Lower is better) | 0.8456 |
| R2 (Higher is better)  | 0.4491 |

### Preprocessing Stage

In [2]:
import optuna
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [3]:
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

X_test = pd.read_csv('./data/X_test.csv')
y_test = pd.read_csv('./data/y_test.csv')

In [4]:
X_train.head()

,latitude,longitude,land_use_label,distance_to_waterbody,distance_to_open_space,subzone,planning_area,region,elevation,temp_2024_04_07_min,...,built-up,bare / sparse vegetation,snow and ice,permanent water bodies,herbaceous wetland,mangroves,moss and lichen,min_ndvi,mean_ndvi,max_ndvi
0,1.327345,103.776261,ROAD,0.005491,0.000305,HOLLAND ROAD,BUKIT TIMAH,CENTRAL REGION,34,28.880736,...,128,1,0,1,0,0,0,0.1176063463,0.2107233339,0.3355351585
1,1.362310,103.885041,RESIDENTIAL,0.002163,0.002288,KOVAN,HOUGANG,NORTH-EAST REGION,14,33.603571,...,183,1,0,0,0,0,0,0.06873453002,0.1237388913,0.1772913102
2,1.304792,103.740678,BUSINESS 2,0.001660,0.001437,PENJURU CRESCENT,JURONG EAST,WEST REGION,10,28.880736,...,251,8,0,33,0,0,0,0.03399855502,0.07334574643,0.1149060753
3,1.432131,103.793028,ROAD,0.002688,0.002472,WOODLANDS SOUTH,WOODLANDS,NORTH REGION,32,30.168782,...,-,-,-,-,-,-,-,-,-,-
4,1.303530,103.820861,CIVIC & COMMUNITY INSTITUTION,0.011124,0.004127,RIDOUT,TANGLIN,CENTRAL REGION,17,30.168782,...,63,1,0,0,0,0,0,0.09017470784,0.2076336658,0.3255961435


In [5]:
# Combine X and y to make sure that the oversampling is done correctly
X_train = pd.concat([X_train, y_train], axis=1)
X_test = pd.concat([X_test, y_test], axis=1)

- Drop subzone and planning area columns
- Replace land use label by one hot encoding
- Drop temperature data, since they are not independent variables

In [6]:
X_train.columns

Index(['latitude', 'longitude', 'land_use_label', 'distance_to_waterbody',
       'distance_to_open_space', 'subzone', 'planning_area', 'region',
       'elevation', 'temp_2024_04_07_min', 'temp_2024_04_07_max',
       'temp_2024_04_07_median', 'temp_2024_04_08_min', 'temp_2024_04_08_max',
       'temp_2024_04_08_median', 'temp_2024_04_09_min', 'temp_2024_04_09_max',
       'temp_2024_04_09_median', 'temp_2024_04_10_min', 'temp_2024_04_10_max',
       'temp_2024_04_10_median', 'Total_x', 'HDB Total',
       'Condominiums & Other Apartments', 'Landed Properties_x',
       'Other Dwellings_x', 'Floor_below_60', 'Floor_60-80', 'Floor_80-100',
       'Floor_100-120', 'Floor_above_120', 'Below $1,000', '$1,000 - $1,999',
       '$2,000 - $2,999', '$3,000 - $3,999', '$4,000 - $4,999',
       '$5,000 - $5,999', '$6,000 - $6,999', '$7,000 - $7,999',
       '$8,000 - $8,999', '$9,000 - $9,999', '$10,000 - 10,999',
       '$11,000 - 11,999', '$12,000 - $14,999', '$15,000 & Over', 'tree cover',
 

In [7]:
columns_to_drop = ['land_use_label', 'subzone', 'planning_area', 'region',
       'temp_2024_04_07_min', 'temp_2024_04_07_max',
       'temp_2024_04_07_median', 'temp_2024_04_08_min', 'temp_2024_04_08_max',
       'temp_2024_04_08_median', 'temp_2024_04_09_min', 'temp_2024_04_09_max',
       'temp_2024_04_09_median', 'temp_2024_04_10_min', 'temp_2024_04_10_max',
       'temp_2024_04_10_median']

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

In [8]:
# Remove rows where min_ndvi values is -
X_train = X_train[X_train['min_ndvi'] != '-']
X_test = X_test[X_test['min_ndvi'] != '-']

In [9]:
# Split X and y
y_train = X_train['avg_temp']
X_train = X_train.drop(columns=['avg_temp'])

y_test = X_test['avg_temp']
X_test = X_test.drop(columns=['avg_temp'])

## Model Training

In [11]:
# Scale the features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    # Hyperparameter space definition
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    C = trial.suggest_float('C', 0.1, 100.0, log=True)  # Regularization parameter
    epsilon = trial.suggest_float('epsilon', 0.01, 1.0, log=True)  # Epsilon in the epsilon-SVR model
    
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)  # Degree of the polynomial kernel
    else:
        degree = 3  # Default degree for non-poly kernels

    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])  # Kernel coefficient

    # Create and train the SVR model
    regressor = SVR(kernel=kernel, C=C, epsilon=epsilon, degree=degree, gamma=gamma)
    regressor.fit(X_train_scaled, y_train)

    # Predict and calculate the R2 score
    y_pred = regressor.predict(X_test_scaled)
    score = r2_score(y_test, y_pred)
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

print("Best trial:")
trial = study.best_trial
print(f"  R2 score: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-17 02:27:42,316] A new study created in memory with name: no-name-089bd5f6-433c-40de-90e1-f6d204894209


[I 2024-04-17 02:27:42,851] Trial 0 finished with value: 0.29182290255201304 and parameters: {'kernel': 'linear', 'C': 9.377031994071686, 'epsilon': 0.010161411843487528, 'gamma': 'auto'}. Best is trial 0 with value: 0.29182290255201304.
[I 2024-04-17 02:27:44,995] Trial 1 finished with value: 0.3071906739773078 and parameters: {'kernel': 'linear', 'C': 38.8243026542383, 'epsilon': 0.14047280517120156, 'gamma': 'auto'}. Best is trial 1 with value: 0.3071906739773078.
[I 2024-04-17 02:27:45,035] Trial 2 finished with value: 0.2927584256042195 and parameters: {'kernel': 'rbf', 'C': 2.835893052868514, 'epsilon': 0.011585135284234745, 'gamma': 'auto'}. Best is trial 1 with value: 0.3071906739773078.
[I 2024-04-17 02:27:45,194] Trial 3 finished with value: 0.3006373476311789 and parameters: {'kernel': 'linear', 'C': 2.646269000556378, 'epsilon': 0.20315050866260195, 'gamma': 'auto'}. Best is trial 1 with value: 0.3071906739773078.
[I 2024-04-17 02:27:45,229] Trial 4 finished with value: 0.3

Best trial:
  R2 score: 0.4490217042137056
  Params: 
    kernel: rbf
    C: 2.5835792187344
    epsilon: 0.8159626574541393
    gamma: auto


In [12]:
regressor = SVR(
    kernel='rbf',
    C=2.5835792187344,
    epsilon=0.8159626574541393,
    degree=3,
    gamma='auto'

)

regressor.fit(X_train_scaled, y_train)

y_pred = regressor.predict(X_test_scaled)
score = r2_score(y_test, y_pred)

# Calculate the RMSE
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print(f"RMSE: {rmse}")

# Calculate the R2
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2}")

RMSE: 0.8456627696170189
R2: 0.4490217042137056


### Approach 2: Minimise RMSE

In [13]:
# Scale the features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    # Hyperparameter space definition
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    C = trial.suggest_float('C', 0.1, 100.0, log=True)  # Regularization parameter
    epsilon = trial.suggest_float('epsilon', 0.01, 1.0, log=True)  # Epsilon in the epsilon-SVR model
    
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)  # Degree of the polynomial kernel
    else:
        degree = 3  # Default degree for non-poly kernels

    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])  # Kernel coefficient

    # Create and train the SVR model
    regressor = SVR(kernel=kernel, C=C, epsilon=epsilon, degree=degree, gamma=gamma)
    regressor.fit(X_train_scaled, y_train)

    # Predict and calculate the R2 score
    y_pred = regressor.predict(X_test_scaled)
    score = np.sqrt(np.mean((y_test - y_pred)**2))
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

print("Best trial:")
trial = study.best_trial
print(f"  R2 score: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-17 02:30:21,784] A new study created in memory with name: no-name-5fc05578-149c-4b56-b275-c0facf5b1373


[I 2024-04-17 02:30:21,856] Trial 0 finished with value: 1.2618831255731593 and parameters: {'kernel': 'poly', 'C': 48.369499987633795, 'epsilon': 0.3215984029417114, 'degree': 4, 'gamma': 'scale'}. Best is trial 0 with value: 1.2618831255731593.
[I 2024-04-17 02:30:21,993] Trial 1 finished with value: 1.1074202369705992 and parameters: {'kernel': 'poly', 'C': 91.0002008802646, 'epsilon': 0.010777928091084138, 'degree': 2, 'gamma': 'auto'}. Best is trial 1 with value: 1.1074202369705992.
[I 2024-04-17 02:30:22,007] Trial 2 finished with value: 0.9592741003554258 and parameters: {'kernel': 'poly', 'C': 0.2236608471197491, 'epsilon': 0.8521954642077189, 'degree': 3, 'gamma': 'scale'}. Best is trial 2 with value: 0.9592741003554258.
[I 2024-04-17 02:30:22,044] Trial 3 finished with value: 0.9445494729678493 and parameters: {'kernel': 'linear', 'C': 0.16883472164784438, 'epsilon': 0.2668733288488516, 'gamma': 'auto'}. Best is trial 3 with value: 0.9445494729678493.
[I 2024-04-17 02:30:22,0

Best trial:
  R2 score: 0.8456036134366282
  Params: 
    kernel: rbf
    C: 2.6466199881312584
    epsilon: 0.8097896401949417
    gamma: auto


In [14]:
regressor = SVR(
    kernel='rbf',
    C=2.6466199881312584,
    epsilon=0.8097896401949417,
    degree=3,
    gamma='auto'

)

regressor.fit(X_train_scaled, y_train)

y_pred = regressor.predict(X_test_scaled)
score = r2_score(y_test, y_pred)

# Calculate the RMSE
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print(f"RMSE: {rmse}")

# Calculate the R2
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2}")

RMSE: 0.8456036134366282
R2: 0.4490987860780631
